# MIS: Clase Complementaria 5 (2023-20)

In [1]:
# Cargue de Paquetes
import numpy as np
import pandas as pd
import networkx as nx
import itertools as it
import time
import matplotlib.pyplot as plt
import graph_tools
import random
#import scipy.linalg as linalg 
#py -m pip install scipy==1.8.1

### Función de Generación de Recompensas

In [2]:
# Se genera la función de recompensas
# List_Distributions tiene dentro la información de cada uno de los brazos del bandido multibrazo con len(List_Distributions) brazos
# Reps es la cantidad de veces que se jugará el bandido multibrazo
 
def Generador_Recompensas(List_Distributions:list,reps:int):
    Recompensas = {}
    contador = 0
    for i in List_Distributions:
        if "Bernoulli" in i:
            p = float(i.replace("Bernoulli","").replace("(","").replace(")",""))
            Recompensas[contador] = [np.random.binomial(1,p) for j in range(reps)]
        elif "Binomial" in i:
            n,p = i.replace("Binomial","").replace("(","").replace(")","").split(",")
            Recompensas[contador] =  [np.random.binomial(n,p) for j in range(reps)]
        elif "Uniform" in i:
            a,b = i.replace("Normal","").replace("(","").replace(")","").split(",")
            Recompensas[contador] =  [np.random.uniform(low=a,high=b) for j in range(reps)]
        elif "Normal" in i:
            mu,sigma=i.replace("Normal","").replace("(","").replace(")","").split(",")
            Recompensas[contador] =  [np.random.normal(loc=mu,scale=sigma) for j in range(reps)]
        else:
            raise Exception("Introduzca una distribución válida.")
        contador += 1
    return Recompensas


# Más adelante se generarán las simulaciones. Por ahora nos concentraremos en armar los otros componentes de la práctica de hoy.
# Además, las distribuciones anteriores son generales, pero nos centraremos en la Bernoulli

### Bandidos tipo Bernoulli

In [3]:
# Inicializador de los estimadores Plug-In Q y de Priors de tipo Beta(\alpha,\beta)
# Para Plug-In tipo Q se introducen listas de números. Para Priors tipo Beta se introducen listas de listas de 2 componentes

def Inicializador_Input_Optimizacion(List_Distributions:list,List_Parameters:list,tipo:str):
    l_dist = len(List_Distributions)
    l_param = len(List_Parameters)
    for parameter in List_Parameters:
        if tipo=="Q":
            if isinstance(parameter,(float,int))==False:
                raise Exception("Existen parámetros que no corresponden con su elección de tipo Q")
        elif tipo=="Beta":
            if (isinstance(parameter,(list))==False) or (len(parameter)!=2):
                raise Exception("Existen parámetros que no corresponden con su elección de tipo Beta")
            else:
                if any([((isinstance(k,(int,float))==False) or (k<=0)) for k in parameter]):
                    raise Exception("Parametros inválidos para la distribución Beta")
        else:
            raise Exception('Introduzca un tipo de inicialización válida')
    if l_dist==l_param:
        return List_Parameters
    elif l_param==1:
        return List_Parameters*l_dist
    else:
        raise Exception("Introduzca una listado de parámetros válidos. La lista que introdujo no puede ser procesada")

In [4]:
# Se crea la función de ArgMax necesaria en los modelos Greedy y Epsilon-Greedy

def Optimizador_Arg_Max(List_Expected_Values:list,tipo:str="Q"):
    if len(List_Expected_Values)==0:
        raise Exception("Introduzca una lista no vacía")
    if (isinstance(tipo,(str))==False) or (tipo not in ['Q',"Beta"]):
        raise Exception("Introduzca un tipo válido")
    
    index = [0]
    l = len(List_Expected_Values)
    contador = 0
    if tipo=="Q":
        max = List_Expected_Values[0]
    elif tipo=="Beta":
        alpha,beta = List_Expected_Values[contador]
        max = alpha/(alpha+beta)
    
    while contador<l:

        if tipo=="Q":
            evaluado = List_Expected_Values[contador]
        elif tipo=="Beta":
            alpha,beta = List_Expected_Values[contador]
            evaluado = alpha/(alpha+beta)

        if evaluado>=max:
            if evaluado==max:
                index.append(contador)
            else:
                max=evaluado
                index = [contador]
        contador+=1
    return random.choice(index)

In [5]:
# Se crea la función de selección aleatoria de brazos (K el número de brazos). Se usa distribución uniforme

def Selector_Aleatorio_Brazos(K:int):
    return random.randint(0,K-1)

In [6]:
# Se crea la función de "lanzar la moneda" para explorar/explotar

def Explore_Exploit(epsilon:float):
    return np.random.binomial(1,epsilon)

In [7]:
# Se crea la función que permite ver la realización de la recompensa

def Realizacion_Recompensa(Recompensas:dict,time:int,choice:int):
    return Recompensas[choice][time]

In [8]:
# Se crea la función de sampleo de TS

def Sampling(List_Parameters):
    Sample = []
    for i in List_Parameters:
        alpha,beta=i
        Sample.append(np.random.beta(alpha,beta))
    return Sample

In [9]:
# Actualización de las Q's y parámetros de las Betas según sea el caso

def Actualizador(List_Parameters:list,reward:float,argmax:int,tipo:str,selected:list=[0]):
    if tipo=="Q":
        if len(selected)!=len(List_Parameters):
            raise Exception("El record de elecciones de acciones está mal")
        else:
            Q_present = List_Parameters[argmax]
            if selected[argmax] == 0:
                Q_new = 1
            else:
                Q_new = Q_present+(1/selected[argmax])*(reward-Q_present)
            List_Parameters[argmax] = Q_new
            selected[argmax] += 1
    elif tipo=="Beta":
        alpha_present,beta_present=List_Parameters[argmax]
        List_Parameters[argmax] = [alpha_present+reward,beta_present+1-reward]
    return List_Parameters,selected

### Modelos de aprendizaje

#### Modelos de tipo Q (Básicos)

In [10]:
# QGreedy
def QGreedy(Recompensas,Priors_t_0):
    acciones = []
    pagos = []
    K = len(Recompensas)
    T = len(Recompensas[0])
    t=0
    # Selected (número de veces que se ha seleccionado una acción)
    selected = [0]*K 
    # Estimación del modelo
    Q = Priors_t_0
    while t<T:
        # Elección de la acción
        choice = Optimizador_Arg_Max(Q,"Q")
        # Realización de la recompensa
        reward = Realizacion_Recompensa(Recompensas=Recompensas,time=t,choice=choice)
        # Guardado de las cosas relevantes
        acciones.append(choice)
        pagos.append(reward)
        # Recálculo del modelo
        Q,selected = Actualizador(List_Parameters=Q,reward=reward,argmax=choice,tipo="Q",selected=selected)
        print(choice,reward)
        # Paso siguiente
        t+=1

    return acciones,pagos   


In [11]:
# QEpsilonGreedy
def QEpsilonGreedy(Recompensas,Priors_t_0,epsilon):
    if (isinstance(epsilon,(float))==False) or (0<=epsilon<=1)==False:
        raise Exception('Introduzca epsilon válido')
    
    acciones = []
    pagos = []
    K = len(Recompensas)
    T = len(Recompensas[0])
    t=0
    # Selected (número de veces que se ha seleccionado una acción)
    selected = [0]*K 
    # Estimación del modelo
    Q = Priors_t_0
    while t<T:
        # Elección de la acción
        if Explore_Exploit(epsilon=epsilon) == 1:
            choice = Selector_Aleatorio_Brazos(K=K)
        else:
            choice = Optimizador_Arg_Max(Q,"Q")
        # Realización de la recompensa
        reward = Realizacion_Recompensa(Recompensas=Recompensas,time=t,choice=choice)
        # Guardado de las cosas relevantes
        acciones.append(choice)
        pagos.append(reward)
        # Recálculo del modelo
        Q,selected = Actualizador(List_Parameters=Q,reward=reward,argmax=choice,tipo="Q",selected=selected)
        print(choice,reward)
        # Paso siguiente
        t+=1

    return acciones,pagos   

#### Modelos de tipo Bayesiano con distribuciones Beta y uso de medias

In [12]:
# BetaGreedy
def BetaGreedy(Recompensas,Priors_t_0):
    acciones = []
    pagos = []
    T = len(Recompensas[0])
    t=0
    # Estimación del modelo
    Parameters = Priors_t_0
    while t<T:
        # Elección de la acción
        choice = Optimizador_Arg_Max(Parameters,"Beta")
        # Realización de la recompensa
        reward = Realizacion_Recompensa(Recompensas=Recompensas,time=t,choice=choice)
        # Guardado de las cosas relevantes
        acciones.append(choice)
        pagos.append(reward)
        # Recálculo del modelo
        Parameters,selected = Actualizador(List_Parameters=Parameters,reward=reward,argmax=choice,tipo="Beta")
        print(choice,reward)
        # Paso siguiente
        t+=1

    return acciones,pagos   

In [13]:
# BetaEpsilonGreedy
def BetaEpsilonGreedy(Recompensas,Priors_t_0,epsilon):
    if (isinstance(epsilon,(float))==False) or (0<=epsilon<=1)==False:
        raise Exception('Introduzca epsilon válido')
    
    acciones = []
    pagos = []
    K = len(Recompensas)
    T = len(Recompensas[0])
    t=0
    # Estimación del modelo
    Parameters = Priors_t_0
    while t<T:
        # Elección de la acción
        if Explore_Exploit(epsilon=epsilon) == 1:
            choice = Selector_Aleatorio_Brazos(K=K)
        else:
            choice = Optimizador_Arg_Max(Parameters,"Beta")
        # Realización de la recompensa
        reward = Realizacion_Recompensa(Recompensas=Recompensas,time=t,choice=choice)
        # Guardado de las cosas relevantes
        acciones.append(choice)
        pagos.append(reward)
        # Recálculo del modelo
        Parameters,selected = Actualizador(List_Parameters=Parameters,reward=reward,argmax=choice,tipo="Beta")
        print(choice,reward)
        # Paso siguiente
        t+=1

    return acciones,pagos   


#### Modelo de tipo bayesiano con distribuciones Beta y Thompson Sampling 

In [14]:
# Modelo de tipo bayesiano con distribuciones Beta y Thompson Sampling 
def BetaTS(Recompensas,Priors_t_0):
    acciones = []
    pagos = []
    T = len(Recompensas[0])
    t=0
    # Estimación del modelo
    Parameters = Priors_t_0
    while t<T:
        # Sampleo
        Sampleo = Sampling(List_Parameters = Parameters)
        # Elección de la acción
        choice = Optimizador_Arg_Max(Sampleo)
        # Realización de la recompensa
        reward = Realizacion_Recompensa(Recompensas=Recompensas,time=t,choice=choice)
        # Guardado de las cosas relevantes
        acciones.append(choice)
        pagos.append(reward)
        # Recálculo del modelo
        Parameters,selected = Actualizador(List_Parameters=Parameters,reward=reward,argmax=choice,tipo="Beta")
        print(choice,reward)
        # Paso siguiente
        t+=1

    return acciones,pagos  

# Ojo, no tiene sentido proponer un Thompson Sampling Greedy y Epsilon Greedy, pues precisamente el sampling es lo que permite la exploración.